# Reconstructing wildfire propagation - An example

This notebook is a step-by-step guide towards the reconstruction of wildfire propagation by combining a burned area database and active fire detection points. Multiple methods were sequentially applied to pre-process the two datasets, combine their inputs and create individual raster files, which contain the fire propgation intervals. If necessary, the starting point of the fire can be approximated as well. 

In this example, the EFFIS burned area database is used. It is free to download from the website of the [European Forest Fire Information System (EFFIS)](https://forest-fire.emergency.copernicus.eu/applications/data-and-services). 
The database is provided in an shp-format and includes wildfire events from different countries in Europe, North Africa, and the Middle East. It ranges back to the year 2000, where 13 countries started to provide the necessary information. 
The number of participating countries grew steadily over the years, and today, 50 countries contribute the data of wildfire events happening in their habitat. 
Every fire event can be located with a specific ID and its start- and end-date is given according to the information provided by the countries authorities. 
Since wildfires recordings depend on national authorities and ministries, information on start- and end-date can be of varying accuracy. 

The pre-processed active fire product from the [VIIRS sensor](https://www.earthdata.nasa.gov/learn/find-data/near-real-time/firms/viirs-i-band-375-m-active-fire-data) was used to include the active fire detection points. This data is provided by NASAs Fire Information for Resource Management System (FIRMS) and can be [downloaded](https://firms.modaps.eosdis.nasa.gov/download/) in varying formats (in this example, the csv-format is used) for different years, countries or self-defined areas. 
Other active fire products can be used as well, and especially MODIS is often applied to detect thermal anomalies all over the world. The relevant MODIS product can also be downloaded from the linked website. 

In this example, fire propagation is recunstructed for all wildfires in Italy in the year 2019 (or more specfic: all events from this year that are recoded in the EFFIS database). Thereto, the specific burned areas are selected from the EFFIS database and combined with VIIRS active fire detection data from the same year. The resulting fire propagation images are saved as .tif-files in a specified output folder. To code along, the necessary data needs to be downloaded first.  

### 1. Importing the necessary libraries

While importing most necessary libraries can be done normally (e.g., using conda-forge and a specified conda environment), GDAL is used in geopandas, rasterio and rioxarray, which can sometimes lead to conflicts. This can be circumvented by following a specific order of installation. In the presented example, we used the Python Version 3.11.9 and installed the aforementioned packages in the following order:
1. conda install -c conda-forge gdal
2. conda install -c conda-forge geopandas rasterio
3. conda install -c conda-forge rioxarray

In [ ]:
# import necessary libraries
import pandas as pd
import geopandas as geo
import xarray
import rioxarray as rio
from osgeo import osr, ogr, gdal
import numpy as np
import rasterio
from rasterio import Affine, features
from rasterio.mask import mask
from rasterio.enums import MergeAlg
import os
import matplotlib.pyplot as plt
from shapely.ops import unary_union
from shapely.geometry import Polygon, Point, mapping, box
from scipy.interpolate import griddata
import scipy.ndimage as ndi
from scipy.spatial import distance_matrix
from scipy.spatial.distance import cdist
import datetime
import concurrent
import ee

### 2. Import the developed modules for calculating fire propagation

In [ ]:
from fire_prop import active_fire_detection, fire_progression, artifacts, check_fire_progression

### 3. Import burned area database

Several columns from the burned area geodataframe are necessary to run the functions that reconstruct fire propagation. Depending on the burned area dataset that is used, the relevant columns can have different names. These names need to be set during class initialization.

In [ ]:
# import the burned area database (in shp-format) as a geopandas dataframe
effis_burnt_area = geo.read_file(".../modis.ba.poly.shp")

The EFFIS dataset for burned areas includes the following columns:

In [ ]:
effis_burnt_area.head()

The important columns include the start- and end-date of a fire, as well as the column that includes the ID of the fire polygon. The fire dates are needed to find the corresponding active fire detections, the ID is used to reference the resulting raster files to their specific fire event.
Depending on the dataset, only the start-date is recorded, in which case the same column is used for both, start and end of the fire. 

Concerning the EFFIS database, the relevant columns are named 'FIREDATE', 'LASTUPDATE'and 'id'. The column names are saved as variables.

In [ ]:
# set important column names of the EFFIS data
col_name_start_fire = 'FIREDATE'
col_name_end_fire = 'LASTUPDATE'
col_name_id = 'id'

To avoid any problems concerning date conventions, the date formats are unified.

In [ ]:
# unify the date columns of start- and end-date in the dataframe
effis_burnt_area[col_name_start_fire] = pd.to_datetime(effis_burnt_area[col_name_start_fire], format='mixed')
effis_burnt_area[col_name_end_fire] = pd.to_datetime(effis_burnt_area[col_name_end_fire], format='mixed')

The column that includes the country is also saved as a variable

In [ ]:
col_name_country = 'COUNTRY'

### 4. Import active fire detection points

Similar to the burned area database, several columns are necessary to run the functions of the 'active_fire_detection' class. Investigating the imported data is therefore an important first step.

In [ ]:
# Read the VIIRS active fire detection file (in .csv-format) for Italy and the year 2019 as a pandas dataframe
viirs_sel_country_and_year = pd.read_csv('.../viirs-snpp_2019_Italy.csv')
# Print the head of the dataframe
viirs_sel_country_and_year.head()

The important columns of the active fire file include the acquisition time, the acquisition date, as well as the latitude and the longitude of the active fire detection points. 

Concerning VIIRS active fire detection, the relevant columns are named 'acq_time', 'acq_date'. 'latitude', and 'longitude'. The column names are saved as variables.

In [ ]:
# set important column names of the VIIRS data
acq_time = "acq_time"
acq_date = "acq_date"
latitude = "latitude"
longitude = "longitude"

### 5. Process active fire detection points

The file with active fire detections includes all active fire detection points of the chosen country and the chosen year. It is therefore necessary to reduce the file to fit the selected burned area geographically and temporally. 
Thereto, a new instance of the active_fire_detection class needs to be initialized. It includes the necessary functions for processing the active fire detection data. 

In [ ]:
# Initialize a new instance of the active_fire_detection using the formerly defined column names of both datasets
act_fire = active_fire_detection(acq_time, acq_date, latitude, longitude, col_name_id, col_name_start_fire, col_name_end_fire)

# Use modify_viirs to convert the active fire detection dataframe to a geopandas dataframe 
viirs_gdf = act_fire.modify_viirs(viirs_sel_country_and_year)

# create subset of the EFFIS database that overlaps with the active fire detection file in time and space
# start with time (i.e., year)
effis_burnt_area_sel_year = effis_burnt_area[~((effis_burnt_area[col_name_start_fire] <= '2019-01-01') | (effis_burnt_area[col_name_start_fire] >= f'2019-12-31'))]
# continue with space (i.e., country)
effis_burnt_area_subset = effis_burnt_area_sel_year[(effis_burnt_area_sel_year[col_name_country] == 'IT')]
effis_burnt_area_subset = effis_burnt_area_subset[~pd.isnull(effis_burnt_area_subset[col_name_start_fire])]

# unify crs of both datasets
effis_gdf_reprj = effis_burnt_area_subset.to_crs(3035)
viirs_gdf_reprj = viirs_gdf.to_crs(3035)

Buffer all EFFIS burnt area polygons in which to search for active fire detections. The buffered geometry is added to the geopandas dataframe of EFFIS.
Naturally, this buffer zone should match the resolution of the active fire detection sensor.
Here, to match the VIIRS resolution, a 200m buffer zone was chosen 

In [ ]:
distance = 200
effis_gdf_reprj["buffer"] = effis_gdf_reprj.buffer(distance)

The EFFIS wildfire database contains a lot of short (< 2 days) and/or small (<10ha) fires. 
Oftentimes, these fires cannot be detected with the available active fire detection sensors.
Therefore, many fire polygons in the database do not have any matching points in the active fire file
The following code finds all polygons for which active fire detection worked 
and creates a dataset of all fires that span multiple days
save their ID for subsetting the original data

In [ ]:
different_dates = []
fire_id = []
start_date = []
end_date = []
all_dates = []
for i in range(len(effis_gdf_reprj)):
    
    selected_poly = effis_gdf_reprj.iloc[i]
    
    # find_viirs_for_effis is used to search for active fire detections that match the selected polygon of the database
    # Don't forget to set day_thresh according to your needs. For this database, 4 days were found to work well
    pot_points_to_sel_fire = act_fire.find_viirs_for_effis(selected_poly, viirs_gdf_reprj, 4)
    
    if len(pot_points_to_sel_fire)>0:
        
        # process the found active fire points 
        pot_points_to_sel_fire = act_fire.combine_sat_paths(pot_points_to_sel_fire, 2, 3)          
        
        # save metadata of the wildfire polygons of the database for which a active fire detection worked 
        different_dates.append(len(np.unique(pot_points_to_sel_fire["time_in_seconds"])))
        fire_id.append(selected_poly["id"])
        start_date.append(min(pot_points_to_sel_fire["datetime_grouped"]))
        end_date.append(max(pot_points_to_sel_fire["datetime_grouped"]))
        all_dates.append(np.unique(pot_points_to_sel_fire["datetime_grouped"]))
        
# rearange the date format for later
all_dates_new = []
for k in range(len(all_dates)):
    dates_lists = all_dates[k]
    sel_measurement = []
    for i in range(len(dates_lists)):
        sel_measurement.append(str(dates_lists[i])[0:16].replace('-','_').replace(':','_') + '_00')
             
    all_dates_new.append(sel_measurement)

With the caclualted information, the EFFIS database can be reduced to only include fires that have active fire detection points for at least two consecutive dates.

In [ ]:
# create index for all fire polygons of the database that were detected at at least two different dates 
idx_dates = (np.where(np.array(different_dates) > 1)[0])
fire_idx_dates = np.array(fire_id)[idx_dates]

# select all fire polygons using the index
effis_to_use = act_fire.select_subset_using_id(effis_burnt_area_subset, fire_idx_dates)
effis_to_use = effis_to_use.to_crs(3035)

The final dataset is supposed to include equally sized raster files, since most use cases require equally sized images. Therefore, a new gepopandas dataframe with empty but equally sized rectangles is created. Every rectangle centers around a fire event. To restrict the size of the rectangles (and with it the size that is needed to save the data physically on a disk), width and height of the rectangle are adapted. This can be done according to the maximum fire size of all fires, or it can be set to not exceed a specific fire size.
In this example, we set the maximum size of the rectangle according to the largest fires below a size of 10'000 ha. Not restricting the can cause the size of Sentinel-2 data to explode.

In [ ]:
polygon_extends = effis_to_use.bounds

polygon_extends['width'] = np.abs(polygon_extends["maxx"] - polygon_extends["minx"])
polygon_extends['height'] = np.abs(polygon_extends["maxy"] - polygon_extends["miny"])
polygon_extends['centroid'] = effis_to_use.centroid
polygon_extends['burned_ha'] = effis_to_use.area/10000
polygon_extends['country'] = effis_to_use[col_name_country]
polygon_extends['date'] = (effis_to_use[col_name_start_fire]).astype(str)
polygon_extends['id'] = effis_to_use[col_name_id]
polygon_extends['start_date'] = (np.array(start_date)[np.isin(np.array(fire_id),effis_to_use[col_name_id])]).astype(str)
polygon_extends['end_date'] = (np.array(end_date)[np.isin(np.array(fire_id),effis_to_use[col_name_id])]).astype(str)

# according to copernicus (https://climate.copernicus.eu/esotc/2021/wildfires), wildfire > 10'000 ha are 'critical fires'
# these are removed here
polygon_extends = polygon_extends[polygon_extends['burned_ha']<=10000]

print(f"number of fires smaller than 10000ha: {len(polygon_extends)}")
print(f"maximum width of investigated fires: {max(polygon_extends['width'])}")
print(f"maximum hight of investigated fires: {max(polygon_extends['height'])}")

# create length and height of all the equally sized polygons
width_to_centroid = (max(polygon_extends['width']) / 2) + 150
heigth_to_centroid = (max(polygon_extends['height']) / 2) + 150 
# create new geodataframe with the center of every fire polygon
polygon_extends_gdf = geo.GeoDataFrame(
                            polygon_extends, geometry=polygon_extends["centroid"], crs="EPSG:3035"
                        )

# subset the effis dataset again, according to the thresholds in fire size
effis_to_use = act_fire.select_subset_using_id(effis_to_use, polygon_extends_gdf["id"])

# with the centroid, create the rectangular, equally sized geometries for every fire polygon
max_width_new_polygon = polygon_extends_gdf.geometry.x + width_to_centroid
min_width_new_polygon = polygon_extends_gdf.geometry.x - width_to_centroid

max_height_new_polygon = polygon_extends_gdf.geometry.y + heigth_to_centroid
min_height_new_polygon = polygon_extends_gdf.geometry.y - heigth_to_centroid

# create the final geodataframe that contains all equally sized rectangles
to_poly =[]
for i in range(len(polygon_extends_gdf)):
    # Create a rectangular polygon using Shapely's box function
    rectangular_polygon = box(min_width_new_polygon.iloc[i], min_height_new_polygon.iloc[i], max_width_new_polygon.iloc[i], max_height_new_polygon.iloc[i])
    to_poly.append(rectangular_polygon)
gdf_poly = geo.GeoDataFrame(polygon_extends, geometry = to_poly, crs=3035)
gdf_poly = gdf_poly.drop(["centroid"], axis=1)

### 6. Create the final fire propagation dataset

In the final step, fire propagation is reconstructed based on the burned area polygons and the pre-processed active fire detection points of the selected fire. For every individual acquisition time of the sensor that captures the active fires, a raster file with the respective burned area propagation at the time of acquisition is calculated and saved to disk (in the tif-format). Width and height of all raster files are taken from the formerly created geodandas dataframe. They are linked by using the unique id of every fire event. 

Before the raster files of an event are saved, potential artifacts are removed and the propagation is checked for plausibility. Additionally, the starting point of a fire is approximated and saved in a new geodataframe. The starting point has the same id as the corresponding fire event. 

In [ ]:
# define an empty geodataframes, in which 
final_points_std = geo.GeoDataFrame()

# initialize new instances of the classes that are necessary to calculate fire progression, 
# remove progression artifacts, and check for plausible fire propagation 
fire_to_process = fire_progression('time_in_seconds', 'geometry')
rem_art = artifacts('time_in_seconds')
check_fire_prog = check_fire_progression()

# specify the output path. The resulting raster files will be saved in this path (in the tif-format)
out_path = "C:/Users/smuelle4/treeads/creating_reference/fire_propagation_tifs_test/italy"

# iterate over all fire events that have at least two different active fire detection dates
# for every event, fire propagation is recunstruced individually
for i in range(len(gdf_poly)):
    
    # the burned area of every event is selected iteratively from the reduced burned area database
    selected_rectangle = gdf_poly.iloc[i]
    selected_poly = effis_gdf_reprj[(effis_gdf_reprj[col_name_id] == selected_rectangle['id'])].iloc[0]
    
    # find_viirs_for_effis is used to search for active fire detections that match the selected polygon of the database
    # day_thresh needs to be set according to your needs. The defined number of days are substracted/added to the start-/end-date of your fire event.
    # This is necessary, because the dates reported in the burned area database are often not perfectly accurate, 
    # mostly underestimating the start- and end-date of a fire event. 
    # For the EFFIS database, 4 days worked best
    pot_points_to_sel_fire = act_fire.find_viirs_for_effis(selected_poly,viirs_gdf_reprj,4)
    
    if len(pot_points_to_sel_fire)>0:
        
        # process the active fire detection points that are linked to the selected burned area
        pot_points_to_sel_fire = act_fire.combine_sat_paths(pot_points_to_sel_fire, 2, 3)     

        if len(np.unique(pot_points_to_sel_fire["time_in_seconds"])) > 1:
            
            # fill_poly_with_hulls is used to reconstruct fire propagation  
            data_xr_clip = fire_to_process.fill_poly_with_hulls(selected_poly, pot_points_to_sel_fire, effis_to_use.crs)

            # these functions are used to remove potential artifacts induced by the k-nearest neighbor algorithm
            data_xr_clip = rem_art.remove_single_pixel(data_xr_clip)
            data_xr_clip = rem_art.check_for_islands(data_xr_clip)
            data_xr_clip = rem_art.rem_artifacts(data_xr_clip, pot_points_to_sel_fire, 150)
            
            if len(np.unique(data_xr_clip))>1:
                
                # fire_prog_with_iterations is used to check for plausible fire propagation#
                # if initial_fire_cluster = 0, fire propagation is not plausible
                # if initial_fire_cluster = 1, fire propagation is plausible and exactly one starting area exists
                # if initial_fire_cluster > 1, fire propagation is plausible, but multiple starting areas exist. 
                # In this case, the largest area is used to approximate the starting point
                fire_progr, initial_fire_cluster = check_fire_prog.fire_prog_with_iterations(data_xr_clip,3)

                # if fire propagation is plausible (i.e. initial_fire_cluster >= 1), raster files are saved for every date of fire propagation.
                # the number initial_fire_cluster
                if len(initial_fire_cluster)>=1:
                    
                    # find the largest starting area
                    size_cluster = []
                    for k in range(len(initial_fire_cluster)):
                        size_cluster.append(len(np.where(initial_fire_cluster[k]==True)[0]))   
                    
                    # get all the individual fire propagation dates    
                    unique_seconds = np.unique(data_xr_clip)[~np.isnan(np.unique(data_xr_clip))]
                    dates_lists = np.unique(pot_points_to_sel_fire["datetime_grouped"].iloc[np.where(np.isin(pot_points_to_sel_fire["time_in_seconds"], unique_seconds))[0]])

                    # convert every date object to a string and unify the format 
                    dates_str = []
                    for i in range(len(dates_lists)):
                        dates_str.append(str(dates_lists[i])[0:16].replace('-','_').replace(':','_') + '_00')    
                             
                    # approximate the ignition point and add it to the geodataframe                         
                    ign_std = fire_to_process.calc_ignition_point(data_xr_clip, initial_fire_cluster[np.argmax(size_cluster)])
                    for_starting_points_std = fire_to_process.create_new_point(selected_poly, ign_std[0], ign_std[1], 3035)
                    final_points_std = pd.concat([final_points_std,for_starting_points_std])
                    
                    # fire_prog_raster_ident_size iteratively saves one raster for every step of fire propagation in the defined output path
                    fire_to_process.fire_prog_raster_ident_size(data_xr_clip, selected_rectangle, 30, 3035, dates_str, out_path)